# A.3 Broadcasting（广播）

广播（Broadcasting）指的是有不同形状的数组是如何进行运算的。尽管很强大，但却很容易被混淆。一个最简单的例子是把一个标量作用在一个数组上的时候：

In [1]:
import numpy as np

In [2]:
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [3]:
arr * 4

array([ 0,  4,  8, 12, 16])

这里我们说标量4被广播给了其他元素。

例如，我们想要把每一列的数组都减去这一列的平均数（这种做法为demean，译为减去平均数）：

In [4]:
arr = np.random.randn(4, 3)
arr

array([[ 0.19112706, -0.07585204,  1.33214545],
       [-1.51473946,  1.29163832,  0.22056989],
       [ 0.5696424 , -1.11391716,  0.13913336],
       [ 1.2059608 , -0.71633243, -0.2735593 ]])

In [5]:
arr.mean(0)

array([ 0.1129977 , -0.15361583,  0.35457235])

In [6]:
demeaned = arr - arr.mean(0)
demeaned

array([[ 0.07812936,  0.07776379,  0.9775731 ],
       [-1.62773716,  1.44525415, -0.13400246],
       [ 0.4566447 , -0.96030133, -0.21543899],
       [ 1.0929631 , -0.5627166 , -0.62813165]])

In [7]:
demeaned.mean(0) # 下面的结果可以理解为等于0

array([  5.55111512e-17,   5.55111512e-17,   2.77555756e-17])

下图演示了上面的操作原理。对行进行减去平均数是一种广播操作，这种操作需要小心一点。不过只要遵守规则，广播可以把低维的值广播到一个数组上的任何维度（比如在一个二维数组中，让每一行减去那一行的平均值）。

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/rov3s.png)

**The Broadcasting Rule**（广播规则）

Two arrays are compatible for broadcasting if for each trailing dimension (i.e., starting from the end) the axis lengths match or if either of the lengths is 1. Broadcasting is then performed over the missing or length 1 dimensions.

要想让两个数组进行广播，在每个结尾的维度上，轴的长度是匹配的，或者长度为1。然后广播会在缺失的，或长度为1的维度上进行。

考虑一下上面的例子，假设我们想要减去每一列的平均值。因为`arr.mean(0)`长度为3，而且arr的尾部维度也是3，二者匹配，所以是能沿着轴为0进行广播的。根据上面的规则，要想沿着轴为1减去平均值（即每一行减去当前行的平均值），最小的数组形状必须是（4，1）：

In [8]:
arr

array([[ 0.19112706, -0.07585204,  1.33214545],
       [-1.51473946,  1.29163832,  0.22056989],
       [ 0.5696424 , -1.11391716,  0.13913336],
       [ 1.2059608 , -0.71633243, -0.2735593 ]])

In [10]:
row_means = arr.mean(1)
row_means

array([ 0.48247349, -0.00084375, -0.13504713,  0.07202302])

In [11]:
row_means.shape

(4,)

In [12]:
row_means.reshape((4, 1))

array([[ 0.48247349],
       [-0.00084375],
       [-0.13504713],
       [ 0.07202302]])

In [14]:
demeaned = arr - row_means.reshape((4, 1))
demeaned

array([[-0.29134643, -0.55832553,  0.84967196],
       [-1.51389571,  1.29248207,  0.22141364],
       [ 0.70468953, -0.97887003,  0.2741805 ],
       [ 1.13393778, -0.78835545, -0.34558232]])

In [16]:
demeaned.mean(1) # 下面的数字可以理解为0

array([ -3.70074342e-17,  -2.77555756e-17,   1.85037171e-17,
         1.85037171e-17])

上面操作的示意图：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/7cc17.png)

下面是把一个二维数组加到一个三维数组上，沿着轴为0：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/sefsx.png)

# 1 Broadcasting Over Other Axes（在其他轴上进行广播）

在高维数组上进行广播要小心一些，不然就会得到下面这样的错误：

In [17]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

根据广播的规则，在一个小数组上，广播的维度必须是1。下面的例子里，把维度从(4,)变为了(4, 1):

In [18]:
arr - arr.mean(1).reshape((4, 1))

array([[-0.29134643, -0.55832553,  0.84967196],
       [-1.51389571,  1.29248207,  0.22141364],
       [ 0.70468953, -0.97887003,  0.2741805 ],
       [ 1.13393778, -0.78835545, -0.34558232]])

对于三维的数组，只需要把数据变形为合适的形状即可。下图展示了如何在一个三位数组上的每一个维度进行广播：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/f25u0.png)

所以有一个问题，就是要给长度为1的数组添加一个轴，才能用于广播。使用reshape是一种方法，但是插入一个轴需要构建一个元组用来指明新的形状，这个比较玛法。因此，numpy数组提供了一个特别的语法，通过索引来插入新的轴。这里使用np.newaxis属性：

In [24]:
arr = np.zeros((4, 4))
arr

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [25]:
arr_3d = arr[:, np.newaxis, :]
print(arr_3d.shape)
print(arr_3d)

(4, 1, 4)
[[[ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]]]


In [22]:
arr_1d = np.random.normal(size=3)
arr_1d

array([ 1.13938971,  0.36583158, -2.66511773])

In [23]:
arr_1d[:, np.newaxis]

array([[ 1.13938971],
       [ 0.36583158],
       [-2.66511773]])

In [26]:
arr_1d[np.newaxis, :]

array([[ 1.13938971,  0.36583158, -2.66511773]])

如果我们有一个三维数组，想要在轴2上减去平均值，我们需要这么做：

In [28]:
arr = np.random.randn(3, 4, 5)
depth_means = arr.mean(2)
depth_means

array([[-0.76576042, -0.34515386, -0.29435899, -0.6558081 ],
       [ 0.33125031,  0.45033654, -0.67248223,  0.51666093],
       [-0.63660478, -0.4296203 ,  0.44219882,  0.32036302]])

In [29]:
depth_means.shape

(3, 4)

In [30]:
demeaned = arr - depth_means[:, :, np.newaxis]
demeaned.mean(2) # 下面的结果可以理解为0

array([[ -8.88178420e-17,   4.44089210e-17,   3.33066907e-17,
          4.44089210e-17],
       [ -1.66533454e-17,  -1.11022302e-17,   1.11022302e-16,
          6.66133815e-17],
       [ -6.66133815e-17,   0.00000000e+00,   0.00000000e+00,
          4.44089210e-17]])

我们可能会奇怪是否可以在不牺牲性能的同时，在一个轴上进行减去中平均值的操作。有，但是需要一些特别的索引方法：

In [31]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    
    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

# 2 Setting Array Values by Broadcasting（通过广播设置数组值）

上面讲到的广播规则，同样可以适用于通过索引设置值。比如：

In [32]:
arr = np.zeros((4, 3))
arr

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [34]:
arr[:] = 5
arr

array([[ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.],
       [ 5.,  5.,  5.]])

然而，如果我们有一个想要设置为列的一维数组，只要形状符合就能做到：

In [35]:
col = np.array([1.28, -0.42, 0.44, 1.6])

In [37]:
arr[:] = col[:, np.newaxis]
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [38]:
arr[:2] = [[-1.37], [0.509]]
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])